In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

device = "cuda" if torch.cuda.is_available() else "cpu"

class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()

        self.encoder = nn.Sequential(
            nn.Linear(15, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32)
        )
        self.decoder = nn.Sequential(
            nn.Linear(32, 64),
            nn.ReLU(),
            nn.Linear(64, 128),
            nn.ReLU(),
            nn.Linear(128, 15),
            nn.Sigmoid()
        )

        self.mu = nn.Linear(32, 32)
        self.logvar = nn.Linear(32, 32)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std

    def forward(self, x):
        x = self.encoder(x)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = self.reparameterize(mu, logvar)
        return self.decoder(z), mu, logvar

def loss_function(recon_x, x, mu, logvar):
    MSE = nn.MSELoss(reduction="sum")
    reconstruction_loss = MSE(recon_x, x.view(-1, 15))
    KL_divergence = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return reconstruction_loss + KL_divergence

In [3]:
import pickle
import io
class CPU_Unpickler(pickle.Unpickler):
    def find_class(self, module, name):
        if module == "torch.storage" and name == "_load_from_bytes":
            return lambda b: torch.load(io.BytesIO(b), map_location="cpu")
        else:
            return super().find_class(module, name)


with open('real_matrix_list.pickle', 'rb') as f:
    matrix_list = CPU_Unpickler(f).load()

matrix_tensor = torch.stack(matrix_list).to(device)

In [4]:
data = matrix_tensor

data_min = data.min()
data_max = data.max()
data_normalized = (data - data_min) / (data_max - data_min)

vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

epochs = 100
for epoch in range(epochs):
    vae.train()
    train_loss = 0
    for matrix in data_normalized:
        optimizer.zero_grad()
        recon_batch, mu, logvar = vae(matrix.view(-1, 15))
        loss = loss_function(recon_batch, matrix, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
    print('Epoch: {}, Loss: {:.4f}'.format(epoch, train_loss / len(data_normalized)))

Epoch: 0, Loss: 0.0886
Epoch: 1, Loss: 0.0559
Epoch: 2, Loss: 0.0536
Epoch: 3, Loss: 0.0511
Epoch: 4, Loss: 0.0500
Epoch: 5, Loss: 0.0492
Epoch: 6, Loss: 0.0498
Epoch: 7, Loss: 0.0487
Epoch: 8, Loss: 0.0485
Epoch: 9, Loss: 0.0479
Epoch: 10, Loss: 0.0474
Epoch: 11, Loss: 0.0473
Epoch: 12, Loss: 0.0473
Epoch: 13, Loss: 0.0472
Epoch: 14, Loss: 0.0470
Epoch: 15, Loss: 0.0471
Epoch: 16, Loss: 0.0465
Epoch: 17, Loss: 0.0469
Epoch: 18, Loss: 0.0463
Epoch: 19, Loss: 0.0467
Epoch: 20, Loss: 0.0466
Epoch: 21, Loss: 0.0467
Epoch: 22, Loss: 0.0467
Epoch: 23, Loss: 0.0466
Epoch: 24, Loss: 0.0464
Epoch: 25, Loss: 0.0462
Epoch: 26, Loss: 0.0465
Epoch: 27, Loss: 0.0463
Epoch: 28, Loss: 0.0462
Epoch: 29, Loss: 0.0463
Epoch: 30, Loss: 0.0463
Epoch: 31, Loss: 0.0461
Epoch: 32, Loss: 0.0463
Epoch: 33, Loss: 0.0461
Epoch: 34, Loss: 0.0462
Epoch: 35, Loss: 0.0460
Epoch: 36, Loss: 0.0463
Epoch: 37, Loss: 0.0461
Epoch: 38, Loss: 0.0461
Epoch: 39, Loss: 0.0461
Epoch: 40, Loss: 0.0460
Epoch: 41, Loss: 0.0460
Ep

In [6]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

iris = load_iris()
X, y = iris.data, iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.33)
X_train, X_test, y_train, y_test = (
    torch.tensor(X_train, device=device, dtype=torch.float32),
    torch.tensor(X_test, device=device, dtype=torch.float32),
    torch.tensor(y_train, device=device, dtype=torch.long),
    torch.tensor(y_test, device=device, dtype=torch.long),
)

In [7]:
from sklearn.metrics import accuracy_score
from collections import namedtuple

class IrisClassifier(nn.Module):
    def __init__(self):
        super(IrisClassifier, self).__init__()
        self.linear = nn.Linear(4, 3)

    def forward(self, x):
        x = self.linear(x)
        return x

ModelInfo = namedtuple("ModelInfo", ["state_dict", "matrix", "accuracy"])

NUM_OF_MODELS = 500


generated_matrices = []
with torch.no_grad():
    vae.eval()
    for _ in range(NUM_OF_MODELS):
        z = torch.randn(1, 32).to(device)
        generated_matrix = vae.decoder(z)
        generated_matrices.append(
            generated_matrix.view(3, -1)
        )

generated_state_dicts = []
for matrix in generated_matrices:
    W = matrix[:, :-1]
    b = matrix[:, -1]
    state_dict = {"linear.weight": W, "linear.bias": b}
    generated_state_dicts.append(state_dict)

evaluated_models = []
for state_dict in generated_state_dicts:
    model = IrisClassifier().to(device)
    model.load_state_dict(state_dict)
    model.eval()
    with torch.inference_mode():
        y_pred = model(X_test)
        _, labels = torch.max(y_pred, 1)
        accuracy = accuracy_score(y_test.cpu().numpy(), labels.cpu().numpy())
    evaluated_models.append(
        ModelInfo(state_dict=state_dict, matrix=None, accuracy=accuracy)
    )

for i, model_info in enumerate(evaluated_models):
    print(f"Model {i+1} - Accuracy: {model_info.accuracy}")

Model 1 - Accuracy: 0.98
Model 2 - Accuracy: 0.98
Model 3 - Accuracy: 0.98
Model 4 - Accuracy: 0.98
Model 5 - Accuracy: 0.98
Model 6 - Accuracy: 0.98
Model 7 - Accuracy: 0.98
Model 8 - Accuracy: 0.98
Model 9 - Accuracy: 0.98
Model 10 - Accuracy: 0.98
Model 11 - Accuracy: 0.98
Model 12 - Accuracy: 0.98
Model 13 - Accuracy: 0.98
Model 14 - Accuracy: 0.98
Model 15 - Accuracy: 0.98
Model 16 - Accuracy: 0.98
Model 17 - Accuracy: 0.98
Model 18 - Accuracy: 0.98
Model 19 - Accuracy: 0.98
Model 20 - Accuracy: 0.98
Model 21 - Accuracy: 0.98
Model 22 - Accuracy: 0.98
Model 23 - Accuracy: 0.98
Model 24 - Accuracy: 0.98
Model 25 - Accuracy: 0.98
Model 26 - Accuracy: 0.98
Model 27 - Accuracy: 0.98
Model 28 - Accuracy: 0.98
Model 29 - Accuracy: 0.98
Model 30 - Accuracy: 0.98
Model 31 - Accuracy: 0.98
Model 32 - Accuracy: 0.98
Model 33 - Accuracy: 0.98
Model 34 - Accuracy: 0.98
Model 35 - Accuracy: 0.98
Model 36 - Accuracy: 0.98
Model 37 - Accuracy: 0.98
Model 38 - Accuracy: 0.98
Model 39 - Accuracy: 

In [8]:
generated_matrices

[tensor([[0.6167, 0.8051, 0.2091, 0.1403, 0.7234],
         [0.5800, 0.5161, 0.5373, 0.4052, 0.6718],
         [0.4561, 0.3615, 0.7049, 0.8849, 0.2925]], device='cuda:0'),
 tensor([[0.6183, 0.8017, 0.2080, 0.1396, 0.7224],
         [0.5796, 0.5164, 0.5353, 0.4045, 0.6708],
         [0.4567, 0.3627, 0.7021, 0.8834, 0.2934]], device='cuda:0'),
 tensor([[0.6153, 0.7983, 0.2150, 0.1471, 0.7182],
         [0.5793, 0.5152, 0.5360, 0.4071, 0.6676],
         [0.4569, 0.3645, 0.7006, 0.8781, 0.2981]], device='cuda:0'),
 tensor([[0.6159, 0.8004, 0.2121, 0.1444, 0.7202],
         [0.5791, 0.5158, 0.5363, 0.4061, 0.6688],
         [0.4567, 0.3632, 0.7022, 0.8808, 0.2952]], device='cuda:0'),
 tensor([[0.6153, 0.8030, 0.2119, 0.1423, 0.7224],
         [0.5792, 0.5168, 0.5358, 0.4048, 0.6712],
         [0.4570, 0.3620, 0.7027, 0.8820, 0.2951]], device='cuda:0'),
 tensor([[0.6168, 0.8037, 0.2095, 0.1410, 0.7229],
         [0.5805, 0.5148, 0.5371, 0.4061, 0.6711],
         [0.4554, 0.3616, 0.7049, 0.88

In [9]:
with open("generated_models.pickle", "wb") as f:
    pickle.dump(evaluated_models, f)

with open("generated_matrix_list.pickle", "wb") as f:
    pickle.dump(generated_matrices, f)